# Graph Initialization

In [1]:
import pandas as pd

def initialize_multiplex_hypergraph(w: int, h: int):
    nodes = {1: set(), 2: set(), 3: set()}
    for i in range(w):
        for j in range(h):
            nodes[1].add((i, j))
            if (i + j) % 2 == 1:
                nodes[2].add((i, j))
            if (i + j) % 2 == 0:
                nodes[3].add((i, j))

    hyperedges = {1: [], 2: [], 3: []}
    for i, j in nodes[1]:
        if (i + 1, j) in nodes[1]:
            hyperedges[1].append(((i, j), (i + 1, j)))
        if (i, j + 1) in nodes[1]:
            hyperedges[1].append(((i, j), (i, j + 1)))

    for i, j in nodes[2]:
        candidates = [(i + 1, j + 1), (i + 2, j)]
        if all(pt in nodes[2] for pt in candidates):
            hyperedges[2].append(((i, j), (i + 1, j + 1), (i + 2, j)))

    for i, j in nodes[3]:
        if i % 2 == 0 and j % 2 == 0:
            candidates = [(i + 1, j - 1), (i + 1, j + 1), (i + 2, j)]
            if all(pt in nodes[3] for pt in candidates):
                hyperedges[3].append(((i, j), (i + 1, j - 1), (i + 1, j + 1), (i + 2, j)))

    return nodes, hyperedges

In [2]:
# Example usage
nodes, hyperedges = initialize_multiplex_hypergraph(6, 6)

# Summary print
summary = pd.DataFrame({
    "Layer": ["1 (degree-2)", "2 (degree-3)", "3 (degree-4)"],
    "Activated Hyperedges": [len(hyperedges[1]), len(hyperedges[2]), len(hyperedges[3])]
})
print(summary)

          Layer  Activated Hyperedges
0  1 (degree-2)                    60
1  2 (degree-3)                    10
2  3 (degree-4)                     4


# Percolation Process 

# Measure of Connectivity

In [3]:
from collections import defaultdict, deque
from typing import Dict, Set, Tuple, List

def compute_EGR(
    nodes: Dict[int, Set[Tuple[int, int]]],
    hyperedges: Dict[int, List[Tuple[Tuple[int, int], ...]]],
    source: Tuple[int, int],
    target: Tuple[int, int]
) -> int:
    all_hyperedges = []
    for layer in hyperedges:
        all_hyperedges.extend(hyperedges[layer])

    node_to_hyperedges = defaultdict(set)
    for idx, hedge in enumerate(all_hyperedges):
        for node in hedge:
            node_to_hyperedges[node].add(idx)

    used_hyperedges = set()
    EGR = 0

    def dfs(current, target, visited_nodes, used_hyperedges_local):
        if current == target:
            return []
        for hedge_idx in node_to_hyperedges[current]:
            if hedge_idx in used_hyperedges_local:
                continue
            hedge = all_hyperedges[hedge_idx]
            for next_node in hedge:
                if next_node == current or next_node in visited_nodes:
                    continue
                path = dfs(next_node, target, visited_nodes | {next_node}, used_hyperedges_local | {hedge_idx})
                if path is not None:
                    return [hedge_idx] + path
        return None

    while True:
        path = dfs(source, target, {source}, used_hyperedges)
        if path is None:
            break
        used_hyperedges.update(path)
        EGR += 1

    return EGR

def compute_R(
    nodes: Dict[int, Set[Tuple[int, int]]],
    hyperedges: Dict[int, List[Tuple[Tuple[int, int], ...]]]
) -> float:
    all_hyperedges = []
    for layer in hyperedges:
        all_hyperedges.extend(hyperedges[layer])

    adjacency = defaultdict(set)
    for hedge in all_hyperedges:
        for u in hedge:
            for v in hedge:
                if u != v:
                    adjacency[u].add(v)

    visited = set()
    max_component_size = 0

    for node in set().union(*nodes.values()):
        if node in visited:
            continue
        queue = deque([node])
        component = set()
        while queue:
            curr = queue.popleft()
            if curr in visited:
                continue
            visited.add(curr)
            component.add(curr)
            queue.extend(adjacency[curr] - visited)
        max_component_size = max(max_component_size, len(component))

    total_nodes = len(set().union(*nodes.values()))
    R = max_component_size / total_nodes if total_nodes > 0 else 0.0

    return R

In [5]:
# Example usage
nodes, hyperedges = initialize_multiplex_hypergraph(20, 20)
A = (0, 0)
B = (19, 19)

EGR_val = compute_EGR(nodes, hyperedges, A, B)
R_val = compute_R(nodes, hyperedges)

import pandas as pd
pd.DataFrame([{"EGR (A→B)": EGR_val, "Giant Component Ratio (R)": R_val}])

,EGR (A→B),Giant Component Ratio (R)
0,2,1.0
